<a href="https://colab.research.google.com/github/GavinHacker/recsys_model/blob/master/5_prepare_libsvm_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
#from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
from sklearn.datasets import dump_svmlight_file
from sklearn.preprocessing import OneHotEncoder
import pickle as pkl
from sklearn import preprocessing

## 读取CSV到dataframe

In [0]:
df = pd.read_csv('/content/drive/My Drive/colabnotebook/comment_origin_data_1.csv',sep='\t',encoding='utf-8')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#df_data.to_csv('/content/drive/My Drive/colabnotebook/comment_origin_data.csv', sep='\t', encoding='utf-8')

In [0]:
df.shape

(389689, 29)

In [0]:
df = df.drop_duplicates()

In [0]:
df = df.drop(['Unnamed: 0'], axis=1)

In [0]:
df['ID'].shape

(289261,)

In [0]:
df.shape

(289261, 28)

In [0]:
df = df.drop_duplicates(['ID'])

In [0]:
#df.head().T

In [0]:
#df.head()

In [0]:
df = df.drop(['CONTENT'], axis=1)

In [0]:
#df.head().T

In [0]:
df = df.drop(['ADD_TIME_x', 'ADD_TIME_y'], axis=1)

In [0]:
#df.head().T

In [0]:
df.shape

(289228, 25)

In [0]:
#df.iloc[10000]

In [0]:
#df.tail(10)

In [0]:
df = df.reset_index(drop=True)

In [0]:
#df.tail(10)

In [0]:
#df.head(1).T

In [0]:
df_main = df.drop(['name', 'CREATOR','description','img','ID','NEWDATA','MOVIEID'], axis=1)

In [0]:
df_main.shape

(289228, 18)

In [0]:
#df_main.head().T

In [0]:
#df_main.shape

In [0]:
df_main = df_main.drop(['enable'], axis=1)

In [0]:
#df_main.head().T

In [0]:
#pd.to_datetime(df_main['TIME'], format='%d%b%Y:%H:%M:%S.%f')

In [0]:
datetime.datetime.strptime(df['TIME'][0],'%Y-%m-%d %H:%M:%S').year - 2000

15

In [0]:
df_main = df_main.dropna(subset=['USERID','rcount']).reset_index(drop=True)

## 处理时间辅助函数

In [0]:
index = 0

In [0]:
def process_time(t):
  global index
  index = index + 1
  try:
    return datetime.datetime.strptime(t,'%Y-%m-%d %H:%M:%S').year - 2000
  except:
    print(index)

In [0]:
df_main['TIME_DIS'] = df_main['TIME'].apply(lambda x: process_time(x))

In [0]:
df_main.dropna(subset=['USERID']).shape

(289152, 18)

In [0]:
#df_main.head().T

In [0]:
df_main.shape

(289152, 18)

In [0]:
df_main = df_main.drop(['TIME'], axis=1)

In [0]:
df_main.shape

(289152, 17)

In [0]:
df_main.drop(['USERID'], axis=1).shape

(289152, 16)

In [0]:
#df_main.head().T

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
df_movie = pd.read_sql_query("select * from movie", get_connection())

## 获取各个维度数据的频度统计

In [0]:
def get_dim_dict(df, dim_name):
  type_list = list(map(lambda x:x.split('|') ,df[dim_name]))
  type_list = [x for l in type_list for x in l]
  def reduce_func(x, y):
    for i in x:
      if i[0] == y[0][0]:
        x.remove(i)
        x.append(((i[0],i[1] + 1)))
        return x
    x.append(y[0])
    return x
  l = filter(lambda x:x != None, map(lambda x:[(x, 1)], type_list))
  type_zip = reduce(reduce_func, list(l))
  #type_list = sorted(list(set(type_list)))
  #type_zip = zip(list(range(len(type_list))), type_list)
  #print(len(type_zip))
  #print(type_zip)
  type_dict = {}
  for i in type_zip:
    type_dict[i[0]] = i[1]
  return type_dict

In [0]:
type_dict = get_dim_dict(df_movie, 'type')
actors_dict = get_dim_dict(df_movie, 'actors')
director_dict = get_dim_dict(df_movie, 'director')
trait_dict = get_dim_dict(df_movie, 'trait')

## 把频度统计字典序列化，以备之后的模型使用

In [0]:
with open('/content/drive/My Drive/colabnotebook/actors_dict', 'wb') as f:
  pkl.dump(actors_dict, f)
  
with open('/content/drive/My Drive/colabnotebook/director_dict', 'wb') as f:
  pkl.dump(director_dict, f)

In [0]:
train_y = df_main['RATING']

In [0]:
df_main = df_main.drop(['RATING'], axis=1)
df_main = df_main.drop(['userid'], axis=1)

In [0]:
df_main.head(1).T

,0
USERID,cf2349f9c01f9a5cd4050aebd30ab74f
movieid,10533913
type,剧情|奇幻|冒险|喜剧
actors,艾米·波勒|菲利丝·史密斯|理查德·坎德|比尔·哈德尔|刘易斯·布莱克
region,美国
director,彼特·道格特|罗纳尔多·德尔·卡门
trait,感人|经典|励志
rat,8.7
rmax,5
rmin,2


In [0]:
#df_main = df_main.drop([65416,45287,65414]).reset_index(drop=True)
#df_main = df_main.drop([73432]).reset_index(drop=True)

In [0]:
df_main.shape, train_y.shape

((289152, 15), (289152,))

## 构建训练样本的字典形式，准备之后One-hot编码和稀疏矩阵构建

In [0]:
invalid_data_list = []

In [0]:
data_dict_list = []
for i in df_main.index:
  _dict = {}
  is_invalid = False
  #type
  for s_type in df_main.iloc[i]['type'].split('|'):
    _dict[s_type] = 1
  #actors
  for s_actor in df_main.iloc[i]['actors'].split('|'):
    if not s_actor in actors_dict:
      print('invalid data index is ' + str(i))
      invalid_data_list.append(i)
      is_invalid = True
      break
    if actors_dict[s_actor] < 2:
      _dict['other_actor'] = 1
    else:
      _dict[s_actor] = 1
  if is_invalid == True:
    continue;
  #regios
  _dict[df_main.iloc[i]['region']] = 1
  #userid ...
  _dict[df_main.iloc[i]['USERID']] = 1
  _dict[str(df_main.iloc[i]['movieid'])] = 1
  _dict['rat'] = df_main.iloc[i]['rat']
  _dict['rmax'] = df_main.iloc[i]['rmax']
  _dict['rmin'] = df_main.iloc[i]['rmin']
  _dict['ravg'] = df_main.iloc[i]['ravg']
  _dict['rcount'] = df_main.iloc[i]['rcount']
  _dict['rsum'] = df_main.iloc[i]['rsum']
  _dict['rmedian'] = df_main.iloc[i]['rmedian']
  _dict['TIME_DIS'] = df_main.iloc[i]['TIME_DIS']
  #
  #director
  for s_director in df_main.iloc[i]['director'].split('|'):
    if director_dict[s_director] < 2:
      _dict['other_director'] = 1
    else:
      _dict[s_director] = 1
  #trait
  for s_trait in df_main.iloc[i]['trait'].split('|'):
    _dict[s_trait] = 1
  data_dict_list.append(_dict)

In [0]:
invalid_data_list

[]

In [0]:
len(data_dict_list)

289152

In [0]:
train_y.shape

(289152,)

## 使用DictVectorizer向量化

In [0]:
v = DictVectorizer()
train_X = v.fit_transform(data_dict_list)

In [0]:
v

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)

In [0]:
train_X.shape, train_y.shape

((289152, 75619), (289152,))

In [0]:
#pd.DataFrame(data=train_X, columns=v.feature_names_).head(1).T

In [0]:
train_X.shape, train_y.shape

((289152, 75619), (289152,))

In [0]:
# def fitpredict_libfm(trainX, trainY, testX, classification=True, rank=8, n_iter=100):
#     encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
#     trainX = encoder.transform(trainX)
#     testX = encoder.transform(testX)
#     print(type(trainX))
#     train_file = '/content/drive/My Drive/colabnotebook/libfm_train.txt'
#     test_file = '/content/drive/My Drive/colabnotebook/libfm_test.txt'
#     with open(train_file, 'w') as f:
#         dump_svmlight_file(trainX, trainY, f=f)
#     with open(test_file, 'w') as f:
#         dump_svmlight_file(testX, numpy.zeros(testX.shape[0]), f=f)
#     task = 'c' if classification else 'r'
#     console_output = !$LIBFM_PATH -task $task -method mcmc -train $train_file -test $test_file -iter $n_iter -dim '1,1,$rank' -out output.libfm
    
#     libfm_pred = pandas.read_csv('output.libfm', header=None).values.flatten()
#     return libfm_pred

## 拆分训练集和测试集

In [0]:
train_X_ = train_X[0:280000]
train_y_ = train_y[:280000]
test_X_ = train_X[280000:]
test_y_ = train_y[280000:]

In [0]:
train_X_.shape, train_y_.shape, test_X_.shape, test_y_.shape

((280000, 75619), (280000,), (9152, 75619), (9152,))

In [0]:
type(train_X_)

scipy.sparse.csr.csr_matrix

In [0]:
## save to libFM file

In [0]:
# train_file = '/content/drive/My Drive/colabnotebook/recsys_train.txt'
# test_file = '/content/drive/My Drive/colabnotebook/recsys_test.txt'

# dump_svmlight_file(train_X_, train_y_, train_file)
# dump_svmlight_file(test_X_, test_y_, test_file)
  
  
#   with open(test_file, 'w') as f:
#       dump_svmlight_file(testX, numpy.zeros(testX.shape[0]), f=f)

## 构建逻辑斯蒂回归训练集和测试集

In [0]:
test_y_lr_ = test_y_.apply(lambda x: 1 if int(x) > 3 else 0)

In [0]:
train_y_lr_ = train_y_.apply(lambda x: 1 if int(x) > 3 else 0)

In [0]:
train_y_lr_.shape, test_y_lr_.shape

((280000,), (9152,))

In [0]:
## save to lr file

In [0]:
# train_file = '/content/drive/My Drive/colabnotebook/recsys_train_lr.txt'
# test_file = '/content/drive/My Drive/colabnotebook/recsys_test_lr.txt'

# dump_svmlight_file(train_X_, train_y_lr_, train_file)
# dump_svmlight_file(test_X_, test_y_lr_, test_file)
  

In [0]:
scaler = preprocessing.MaxAbsScaler()

In [0]:
train_X_scaling = scaler.fit_transform(train_X_)

In [0]:
test_X_scaling = scaler.fit_transform(test_X_)

In [0]:
# save to scaling lr
# train_file = '/content/drive/My Drive/colabnotebook/recsys_train_scaling_lr.txt'
# test_file = '/content/drive/My Drive/colabnotebook/recsys_test_scaling_lr.txt'

# dump_svmlight_file(train_X_scaling, train_y_lr_, train_file)
# dump_svmlight_file(test_X_scaling, test_y_lr_, test_file)

## 转换为libsvm数据，为之后libFM使用，LR也使用此类型数据

In [0]:
train_file = '/content/drive/My Drive/colabnotebook/recsys_train_scaling.txt'
test_file = '/content/drive/My Drive/colabnotebook/recsys_test_scaling.txt'

dump_svmlight_file(train_X_scaling, train_y_, train_file)
dump_svmlight_file(test_X_scaling, test_y_, test_file)

In [0]:
with open('/content/drive/My Drive/colabnotebook/dict2vec', 'wb') as f:
  pkl.dump(v, f)

In [0]:
v_from_pkl = None
with open('/content/drive/My Drive/colabnotebook/dict2vec', 'rb') as f:
  v_from_pkl = pkl.load(f)

In [0]:
predict_x = v_from_pkl.transform({'美国':1})

In [0]:
predict_x.shape

(1, 75619)